**AutoFormer**

python -u run.py \
  --task_name classification \
  --is_training 1 \
  --root_path ./dataset/EthanolConcentration/ \
  --model_id EthanolConcentration \
  --model $model_name \
  --data UEA \
  --e_layers 3 \
  --batch_size 16 \
  --d_model 128 \
  --d_ff 256 \
  --top_k 3 \
  --des 'Exp' \
  --itr 1 \
  --learning_rate 0.001 \
  --train_epochs 100 \
  --patience 10

In [16]:
import torch    
import torch.nn as nn
import torch.fft
from torch.utils.data import Dataset, DataLoader
from torch import optim
import sys
import argparse
from sklearn import metrics

In [2]:
sys.path.insert(0, "../")
from models.Autoformer import Model

In [3]:
from pathlib import Path
import os
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import time
import copy

In [4]:
data_dir = "/fs01/home/chloexq/MIMIC-IV-Data-Pipeline/data/csv"

In [5]:
df = pd.read_csv(Path(data_dir, 'labels.csv'))

In [6]:
train_patient_list, test_patient_list, _, test_y = train_test_split(df.stay_id.astype(str).values, df.label.values, stratify=df.label.values, test_size=0.3, random_state=42)

In [7]:
val_patient_list, test_patient_list = train_test_split(test_patient_list, stratify=test_y, test_size=0.5, random_state=42)

In [8]:
print(len(train_patient_list), len(val_patient_list), len(test_patient_list))

37 8 9


In [9]:
class MIMIC_Dataset(Dataset):
    def __init__(self, data_dir, patient_list):
        self.data_dir = data_dir
        self.patient_list = patient_list
        self.labels = pd.read_csv(Path(data_dir, 'labels.csv'))
    def __getitem__(self, index):
        patient = self.patient_list[index]
        df = pd.read_csv(Path(data_dir, patient, 'dynamic.csv'))
        # TODO: Normalize and forward-fill
        data = torch.tensor(df.values).float()
        padding_mask = torch.ones(data.shape[0:1]).float()
        label = torch.tensor(self.labels[self.labels['stay_id'].astype(str) == patient].label.values[0]).long()
        return data, label, padding_mask
    def __len__(self):
        return len(self.patient_list)

In [10]:
train_dataset = MIMIC_Dataset(data_dir, train_patient_list)
val_dataset = MIMIC_Dataset(data_dir, val_patient_list)
test_dataset = MIMIC_Dataset(data_dir, test_patient_list)

In [11]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [12]:
max_seq_len = train_dataset[0][0].shape[0]
feature_size = train_dataset[0][0].shape[1]
print(max_seq_len, feature_size)

73 384


In [17]:
configs = {
    "model_name": 'AutoFormer',
    "task_name": "classification",
    "e_layers": 3,
    "d_model": 128,
    "n_heads": 8,
    "label_len": 2,
    "num_class": 2,
    "pred_len": 0,
    "seq_len": max_seq_len, #TO UPDATE
    "moving_avg": 25,
    "output_attention": "store_true",
    "enc_in": feature_size, #TO UPDATE 
    "embed": "timeF",
    "freq": "h",
    "dropout": 0.1,
    "factor": 1,
    "d_ff": 256,
    "activation": "gelu",
}

In [18]:
args = argparse.Namespace()
d = vars(args)
for key in configs:
    d[key] = configs[key]

In [29]:
def evaluate(loader, model, criterion):
    total_loss = []
    preds = []
    trues = []
    metric_dict = {}
    model.eval()
    with torch.no_grad():
        for i, (batch_x, label, padding_mask) in enumerate(loader):
            batch_x = batch_x.float().to(device)
            padding_mask = padding_mask.float().to(device)
            label = label.to(device)
            outputs = model(batch_x, padding_mask, None, None)
            pred = outputs.detach().cpu()
            loss = criterion(pred, label.long().squeeze().cpu())
            total_loss.append(loss)
            preds.append(outputs.detach())
            trues.append(label)
    total_loss = np.average(total_loss)
    preds = torch.cat(preds, 0)
    trues = torch.cat(trues, 0)
    probs = torch.nn.functional.softmax(preds, dim=0)  # (total_samples, num_classes) est. prob. for each class and sample
    predictions = torch.argmax(probs, dim=1).cpu().numpy()  # (total_samples,) int class index for each sample
    trues = trues.flatten().cpu().numpy()
    if trues.sum() > 0:
        metric_dict['AUC'] = metrics.roc_auc_score(trues, probs.cpu().numpy()[:, 1])
        metric_dict['recall'] = metrics.recall_score(trues, predictions)
    else:
        metric_dict['AUC'] = np.nan
        metric_dict['recall'] = np.nan
    metric_dict['accuracy'] = metrics.accuracy_score(trues, predictions)
    metric_dict['f1'] = metrics.f1_score(trues, predictions)
    metric_dict['precision'] = metrics.precision_score(trues, predictions)
    metric_dict['loss'] = total_loss
    return metric_dict

In [30]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [31]:
model = Model(args).float().to(device)

In [32]:
model_optim = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [33]:
patience = 10
counter = 0
best_val_loss = 999

for epoch in range(100):
    iter_count = 0
    train_loss = []
    model.train()
    epoch_time = time.time()
    for i, (batch_x, label, padding_mask) in enumerate(train_loader):
        iter_count += 1
        model_optim.zero_grad()
        batch_x = batch_x.float().to(device)
        padding_mask = padding_mask.float().to(device)
        label = label.to(device)
        outputs = model(batch_x, padding_mask, None, None)
        loss = criterion(outputs, label.long().squeeze(-1))
        train_loss.append(loss.item())

        if (i + 1) % 100 == 0:
            print("\titers: {0}, epoch: {1} | loss: {2:.7f}".format(i + 1, epoch + 1, loss.item()))
            speed = (time.time() - time_now) / iter_count
            left_time = speed * ((self.args.train_epochs - epoch) * train_steps - i)
            print('\tspeed: {:.4f}s/iter; left time: {:.4f}s'.format(speed, left_time))
            iter_count = 0
            time_now = time.time()
            
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=4.0)
        model_optim.step()

    print("Epoch: {} cost time: {}".format(epoch + 1, time.time() - epoch_time))
    train_loss = np.average(train_loss)
    val_metrics = evaluate(val_loader, model, criterion)
    test_metrics = evaluate(test_loader, model, criterion)
    print("Train Loss: {:.3f} Val Loss: {:.3f} Val Acc: {:.3f} Val AUC: {:.3f} Val Prec: {:.3f} Val Rec: {:.3f} Vali F1: {:.3f}"
          .format(train_loss, val_metrics['loss'], val_metrics['accuracy'], val_metrics['AUC'], val_metrics['precision'], val_metrics['recall'], val_metrics['f1']))
    print("Test Loss: {:.3f} Test Acc: {:.3f} Test AUC: {:.3f} Test Prec: {:.3f} Test Rec: {:.3f} Test F1: {:.3f}"
          .format(test_metrics['loss'], test_metrics['accuracy'], test_metrics['AUC'], test_metrics['precision'], test_metrics['recall'], test_metrics['f1']))
 
    if val_metrics['loss'] < best_val_loss:
        print('Best model val loss {:.3f}'.format(val_metrics['loss']))
        best_model = copy.deepcopy(model)
        best_val_loss = val_metrics['loss']
        counter = 0
    else:
        counter+=1
    print('############################################################################################')
    if counter >= patience:
        print("Early stopping")
        break

Epoch: 1 cost time: 0.7126014232635498
Train Loss: 0.670 Val Loss: 1.744 Val Acc: 0.250 Val AUC: 1.000 Val Prec: 0.143 Val Rec: 1.000 Vali F1: 0.250
Test Loss: 1.564 Test Acc: 0.444 Test AUC: 0.929 Test Prec: 0.286 Test Rec: 1.000 Test F1: 0.444
Best model val loss 1.744
############################################################################################
Epoch: 2 cost time: 0.640775203704834
Train Loss: 0.641 Val Loss: 1.049 Val Acc: 0.500 Val AUC: 1.000 Val Prec: 0.200 Val Rec: 1.000 Vali F1: 0.333
Test Loss: 1.864 Test Acc: 0.556 Test AUC: 0.714 Test Prec: 0.333 Test Rec: 1.000 Test F1: 0.500
Best model val loss 1.049
############################################################################################
Epoch: 3 cost time: 0.696925163269043
Train Loss: 1.591 Val Loss: 0.633 Val Acc: 0.250 Val AUC: 0.714 Val Prec: 0.143 Val Rec: 1.000 Vali F1: 0.250
Test Loss: 1.122 Test Acc: 0.667 Test AUC: 0.571 Test Prec: 0.000 Test Rec: 0.000 Test F1: 0.000
Best model val loss 0.633


In [34]:
save_dir = Path(f"/h/chloexq/Time-Series-Library/save/dev_{args.model_name}-{time.strftime('%b%d-%H-%M')}/")
save_dir.mkdir(parents=True, exist_ok=True)
print(f"save to {save_dir}")
best_model_path = Path(save_dir, 'best_model.pt')
torch.save(best_model.state_dict(), best_model_path)

save to /h/chloexq/Time-Series-Library/save/dev_AutoFormer-Oct21-21-35


In [35]:
model.load_state_dict(torch.load(best_model_path))  

<All keys matched successfully>